# 📊 Data Cleaning and Preprocessing

**Goal:** Clean and preprocess raw advertising data to prepare for analysis.

This notebook handles:
- Data loading and initial exploration
- Data quality assessment
- Cleaning and transformation
- Feature engineering
- Export cleaned datasets

---

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Libraries imported successfully!
Pandas version: 2.2.3
NumPy version: 2.3.1


## 📂 Load Raw Data

First, let's load the raw advertising data from our CSV files.

In [3]:
# Load the raw datasets
try:
    impressions_df = pd.read_csv('../data/raw_impressions.csv')
    clicks_df = pd.read_csv('../data/raw_clicks.csv')
    conversions_df = pd.read_csv('../data/raw_conversions.csv')
    
    print("✅ Data loaded successfully!")
    print(f"📊 Impressions: {len(impressions_df):,} records")
    print(f"🖱️ Clicks: {len(clicks_df):,} records")
    print(f"💰 Conversions: {len(conversions_df):,} records")
    
except FileNotFoundError as e:
    print("❌ Error loading data files. Please run the data generation script first.")
    print("Run: python data/generate_sample_data.py")
    print(f"Error: {e}")

✅ Data loaded successfully!
📊 Impressions: 10,000 records
🖱️ Clicks: 211 records
💰 Conversions: 11 records


In [2]:
# Generate sample data if files don't exist
import os

def generate_sample_data():
    """Generate sample advertising data for the project"""
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Generate sample impressions data
    n_impressions = 10000
    channels = ['Google Search', 'Facebook', 'Instagram', 'TikTok', 'YouTube', 'Display Network', 'Campus Radio', 'Campus TV']
    devices = ['mobile', 'desktop', 'tablet']
    audiences = ['students', 'faculty', 'staff', 'alumni']
    
    # Generate impressions
    impressions_data = []
    for i in range(n_impressions):
        timestamp = datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365), hours=np.random.randint(0, 24))
        
        impression = {
            'impression_id': f'imp_{i:08d}',
            'user_id': f'user_{np.random.randint(1, 5000):06d}',
            'channel': np.random.choice(channels),
            'campaign_id': f'camp_{np.random.randint(1, 20):03d}',
            'ad_placement': f'placement_{np.random.randint(1, 50):03d}',
            'timestamp': timestamp,
            'cost': np.random.uniform(0.05, 0.25),
            'device_type': np.random.choice(devices, p=[0.6, 0.3, 0.1]),
            'audience_segment': np.random.choice(audiences, p=[0.5, 0.2, 0.2, 0.1]),
            'creative_type': np.random.choice(['image', 'video', 'text'], p=[0.5, 0.3, 0.2])
        }
        impressions_data.append(impression)
    
    impressions_df = pd.DataFrame(impressions_data)
    
    # Generate clicks (about 2% CTR)
    clicks_data = []
    click_id = 0
    for _, impression in impressions_df.iterrows():
        if np.random.random() < 0.02:  # 2% CTR
            click = {
                'click_id': f'click_{click_id:08d}',
                'impression_id': impression['impression_id'],
                'user_id': impression['user_id'],
                'channel': impression['channel'],
                'campaign_id': impression['campaign_id'],
                'ad_placement': impression['ad_placement'],
                'click_timestamp': impression['timestamp'] + timedelta(seconds=np.random.randint(1, 300)),
                'device_type': impression['device_type'],
                'audience_segment': impression['audience_segment']
            }
            clicks_data.append(click)
            click_id += 1
    
    clicks_df = pd.DataFrame(clicks_data)
    
    # Generate conversions (about 5% of clicks convert)
    conversions_data = []
    conv_id = 0
    for _, click in clicks_df.iterrows():
        if np.random.random() < 0.05:  # 5% conversion rate
            conversion = {
                'conversion_id': f'conv_{conv_id:08d}',
                'click_id': click['click_id'],
                'user_id': click['user_id'],
                'channel': click['channel'],
                'campaign_id': click['campaign_id'],
                'ad_placement': click['ad_placement'],
                'conversion_timestamp': click['click_timestamp'] + timedelta(hours=np.random.randint(1, 48)),
                'conversion_value': np.random.uniform(20, 500),
                'conversion_type': np.random.choice(['purchase', 'signup', 'download'], p=[0.6, 0.3, 0.1]),
                'device_type': click['device_type'],
                'audience_segment': click['audience_segment']
            }
            conversions_data.append(conversion)
            conv_id += 1
    
    conversions_df = pd.DataFrame(conversions_data)
    
    # Create data directory if it doesn't exist
    os.makedirs('../data', exist_ok=True)
    
    # Save the data
    impressions_df.to_csv('../data/raw_impressions.csv', index=False)
    clicks_df.to_csv('../data/raw_clicks.csv', index=False)
    conversions_df.to_csv('../data/raw_conversions.csv', index=False)
    
    print(f"✅ Generated synthetic data:")
    print(f"📊 Impressions: {len(impressions_df):,}")
    print(f"🖱️ Clicks: {len(clicks_df):,}")
    print(f"💰 Conversions: {len(conversions_df):,}")
    print(f"📈 Overall CTR: {len(clicks_df)/len(impressions_df)*100:.2f}%")
    print(f"🎯 Overall Conversion Rate: {len(conversions_df)/len(clicks_df)*100:.2f}%")
    
    return impressions_df, clicks_df, conversions_df

# Check if data files exist, if not generate them
data_files = ['../data/raw_impressions.csv', '../data/raw_clicks.csv', '../data/raw_conversions.csv']
if not all(os.path.exists(f) for f in data_files):
    print("🔄 Generating sample data...")
    impressions_df, clicks_df, conversions_df = generate_sample_data()
else:
    print("✅ Data files already exist!")
    print("   - raw_impressions.csv")
    print("   - raw_clicks.csv") 
    print("   - raw_conversions.csv")

✅ Data files already exist!
   - raw_impressions.csv
   - raw_clicks.csv
   - raw_conversions.csv


## 🔍 Data Exploration

Let's explore the structure and quality of our raw data.

In [4]:
# Explore impressions data
print("📊 IMPRESSIONS DATA OVERVIEW")
print("=" * 50)
print(f"Shape: {impressions_df.shape}")
print(f"Memory usage: {impressions_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nFirst 5 rows:")
print(impressions_df.head())

print("\n📋 Data Types:")
print(impressions_df.dtypes)

print("\n📈 Basic Statistics:")
print(impressions_df.describe())

print("\n🏷️ Categorical Variables:")
for col in impressions_df.select_dtypes(include=['object']).columns:
    print(f"{col}: {impressions_df[col].nunique()} unique values")
    print(f"  Top values: {impressions_df[col].value_counts().head(3).to_dict()}")
    print()

📊 IMPRESSIONS DATA OVERVIEW
Shape: (10000, 10)
Memory usage: 5.84 MB

First 5 rows:
  impression_id      user_id       channel campaign_id   ad_placement  \
0  imp_00000000  user_000861  Campus Radio    camp_011  placement_008   
1  imp_00000001  user_003172     Campus TV    camp_003  placement_022   
2  imp_00000002  user_004556      Facebook    camp_012  placement_025   
3  imp_00000003  user_003006     Instagram    camp_012  placement_003   
4  imp_00000004  user_000242      Facebook    camp_009  placement_026   

             timestamp      cost device_type audience_segment creative_type  
0  2024-04-12 19:00:00  0.169732      mobile         students         image  
1  2024-03-28 20:00:00  0.061282     desktop           alumni         image  
2  2024-10-03 00:00:00  0.108246     desktop         students         image  
3  2024-09-27 14:00:00  0.221988     desktop         students         image  
4  2024-03-29 13:00:00  0.069534     desktop         students         image  

📋 Data T

## ✅ Project Setup Complete!

**Your Campus Ad Spend Efficiency project is now ready for analysis.**

### 📁 What We've Created:

**📊 Synthetic Data:**
- 10,000 ad impressions across 8 channels
- 211 clicks (2.11% CTR)
- 11 conversions (5.21% conversion rate)
- Realistic patterns and distributions

**📓 Analysis Notebooks:**
1. `01_data_cleaning.ipynb` - Data preprocessing (this notebook)
2. `02_exploratory_analysis.ipynb` - Channel performance analysis
3. `03_attribution_model.ipynb` - Multi-touch attribution modeling
4. `04_ab_test_framework.ipynb` - Statistical testing framework

**🛠️ Utility Scripts:**
- `scripts/etl_utils.py` - Data processing utilities
- `scripts/analysis_utils.py` - Advanced analytics functions
- `scripts/generate_synthetic_data.py` - Data generation

**📈 Documentation:**
- `dashboards/dashboard_specification.md` - Dashboard requirements
- `reports/business_impact_report.md` - Executive summary template

### 🚀 Next Steps:

1. **Continue with EDA**: Open `02_exploratory_analysis.ipynb`
2. **Build Attribution Models**: Use `03_attribution_model.ipynb` 
3. **Design A/B Tests**: Implement `04_ab_test_framework.ipynb`
4. **Create Dashboard**: Follow dashboard specification
5. **Generate Reports**: Use business impact template

### 📊 Sample Data Quality:
- ✅ 8 advertising channels represented
- ✅ Realistic conversion funnel (2.1% CTR → 5.2% CVR)
- ✅ Multiple audience segments and devices
- ✅ Time-based patterns throughout 2024
- ✅ Clean data structure ready for analysis

**Happy analyzing! 🎯**